* Taking 18 s for 200 token generation - 1 image
* used python 3.10

In [1]:
%pip install git+https://github.com/huggingface/transformers accelerate
%pip install qwen-vl-utils[decord]==0.0.8
%pip install pdf2image


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-nv8s5wj5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-nv8s5wj5
  Resolved https://github.com/huggingface/transformers to commit ec7afad60909dd97d998c1f14681812d69a15728
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached accelerate-1.3.0-py3-none-any.whl.metadata (19 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached accelerate-1.3.0-py3-none-any.whl (336 kB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Created wheel for transformers: filename=transformers-4.49.0.dev0-py3-none-any.whl size=10588009 sha256=071a849460a51dc4e480a038fda5201b97bb0eae95dc502105e66664fc2c0341
  Stored in directory: /tmp/pip-ephem-wheel-cache

In [2]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info


model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)


processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [14]:
from pdf2image import convert_from_path
from PIL import Image
images = convert_from_path("/home/project_nova/Abraham Rodriguez OCR_removed.pdf")
# images = [Image.open(image) for image in images_arr]

In [15]:


def generate_message(image):
    return [
        {
                            "role": "system",
                            "content": """You are a highly capable vision-language assistant specializing in extracting text from images and outputting it in Markdown format. 
                            Prioritize identifying all text, including handwritten content, regardless of its style, size, or location in the image. 
                            Highlight handwritten text using **bold** format and ensure no text is missed."""
        },
     {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image,
            },
            {"type": "text", "text": """Extract all visible text from the entire image, including headers, footers, and any text near the edges or corners. 
                                    Pay special attention to identifying handwritten text, even if it is small, faint, or located in less prominent areas. 
                                    Highlight handwritten text using **bold** format. Present all extracted text exactly as it appears, including formatting, line breaks, and special characters."""},
        ],
    }
]

In [22]:

for idx, img in enumerate(images):
    if idx != 2:
        continue
    messages = generate_message(img)

    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")

    generated_ids = model.generate(**inputs, max_new_tokens=3000)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    print(output_text[0])
    with open(f"page_{idx+1}.md", "w") as md_file:
        md_file.write(output_text[0])
    break

AHC Physicians Inc.

Ronald S. Stogryn MD & Eugene Trotta PA
4242 E. Southcross Ste. 16
San Antonio, Texas 78222
Office 210 333-8311
Fax 210 333-0880

October 28, 2019

Patient: Abraham E. Rodriguez

Abraham E. Rodriguez, 39 yr. old male, was involved in a motor vehicle accident on October 09, 2019 while driving when the vehicle he was in was side swiped. He was wearing his seat belt. He was jolted. He felt pain on shoulder, lower back and ankle. He went to see a chiropractor who examined him and initiated physical therapy.

Family History: Reveals it is positive for diabetes and chronic heart disease.

Today on October 28, 2019 he appears in this office for the first time and complains of pain on shoulders, lower back and ankle. Vital Signs: B/P: 137/86 PULSE: 64/min WT: 278 HT: 5'9. Physical Exam: HEENT: Unremarkable. Neck: Movements of neck arouse pain on dorsum accompanied with stiffness radiating down trapezius. Para cervical musculature is tender. PS: 5/10. Thorax: Heart Rate: is